In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
%matplotlib inline 

In [ ]:
competition_name = 'facial-keypoints-detection'

In [ ]:
path = Path.home() / '.kaggle'/ competition_name

In [ ]:
list(path.glob('*'))

In [ ]:
f = lambda x: np.array(x, dtype='i').reshape((96, 96))
df = (pd.read_csv(path/'training.csv')
      .assign(Image=lambda x: x.Image.str.split(' ').apply(f))
     )

In [ ]:
gray_scale_kwargs = dict(cmap='gray', vmin=0, vmax=255)

In [ ]:
coord_cols = [col for col in df.columns.tolist() if col.endswith(('_x', '_y'))]
coord_cols_prefixes = set(col.rpartition('_')[0] for col in coord_cols)
len(coord_cols_prefixes), coord_cols_prefixes

In [ ]:
sample =  df.sample().iloc[0]
pixels = sample.Image

fig,ax = plt.subplots(1)
ax.imshow(pixels, **gray_scale_kwargs)

for coords_prefix in coord_cols_prefixes:
    coords = sample[coords_prefix + '_x'], sample[coords_prefix + '_y']
    ax.scatter(*coords, c='r')

ax.axes.xaxis.set_visible(False), ax.axes.yaxis.set_visible(False)

plt.show()

In [ ]:
np.zeros((96, 96)) + 255

In [ ]:
pixels = np.zeros((96, 96)) + 255

fig,ax = plt.subplots(1)
ax.imshow(pixels, **gray_scale_kwargs)

for coords_prefix in coord_cols_prefixes:
    coords = df[coords_prefix + '_x'].mean(), sample[coords_prefix + '_y'].mean()
    ax.scatter(*coords, c='r')

ax.axes.xaxis.set_visible(False), ax.axes.yaxis.set_visible(False) 

plt.show()

In [ ]:
df.describe()

In [ ]:
df_no_na = df.dropna()
X_train = np.expand_dims(
    np.asarray(df_no_na.loc[:, 'Image'].tolist()),
    axis=-1
)
y_train = df_no_na.drop('Image', axis=1).values

In [ ]:
X_train.shape, y_train.shape

In [ ]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Conv2D, LeakyReLU, GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Sequential

In [ ]:
model = Sequential()

pretrained_model = MobileNet(input_shape=(96, 96, 3), include_top=False, weights='imagenet')
pretrained_model.trainable = True

model.add(Conv2D(3, (1, 1), padding='same', input_shape=(96, 96, 1)))
model.add(LeakyReLU(alpha=0.1))
model.add(pretrained_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.1))
model.add(Dense(30))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.1)

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(20, 10))
df_history = pd.DataFrame(history.history)
df_history[['loss', 'val_loss']].plot(ax=ax[0])
df_history[['accuracy', 'val_accuracy']].plot(ax=ax[1])
ax[0].set_title('Model Loss', fontsize=12)
ax[1].set_title('Model Acc', fontsize=12)
fig.suptitle('Model Metrics', fontsize=18);

In [ ]:
n = 1005
X_sample, y_sample = X_train[n], y_train[n]
yhat_sample = model.predict(X_train)[n]

In [ ]:
pixels = X_sample

fig,ax = plt.subplots(1)
ax.imshow(pixels, **gray_scale_kwargs)

s= 50
ax.scatter(y_sample[0::2], y_sample[1::2], marker='x', c='r', s=s)
ax.scatter(yhat_sample[0::2], yhat_sample[1::2], marker='o', c='g', s=s, alpha=0.7)

ax.axes.xaxis.set_visible(False), ax.axes.yaxis.set_visible(False)

plt.show